<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [4]:
# импортируем библиотеки
import pandas as pd
import psycopg2
import warnings
warnings.filterwarnings("ignore")

import plotly
import plotly.graph_objs as go
import plotly.express as px
from plotly.subplots import make_subplots

# импортируем библиотеку от одного продвинутого студента
import urllib.parse as req

In [6]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)

# Юнит 0. Подготовка к дополнительным исследованиям

Для дополнительных исследований целесообразно подготовить информацию о городах России и их населению. Воспользуемся Википедией.

По состоянию **на 9 марта 2024 года** в России насчитывается 1122 города.

Согласно своду правил «Градостроительство. Планировка и застройка городских и сельских поселений» (СП 42.13330.2016) от Минстроя РФ, города страны классифицированы следующим образом:

* *Крупнейшие* —  с населением свыше 1 миллиона человек
* *Крупные* —  от 250 тысяч человек до 1 миллиона человек (в том числе подкатегории от 250 до 500 тысяч и от 500 тысяч до 1 миллиона человек)
* *Большие* —  от 100 до 250 тысяч человек
* *Средние* —  от 50 до 100 тысяч человек
* *Малые* —  до 50 тысяч человек 

1. Названия городов соответствующих категроий схраним в виде кортежей
2. Сохраняем данные в отдельные DataFrame(Со столбцами:"Название город", "Население")

In [7]:
# функция для предобработки данных:
def get_df_from_viki(list_of_tables, table_index):
    """Функция для обработки таблиц скачанных со страниц Википедии. Необходима конкретно для выгрузки данных о названии города и его населении. Приводит к числовому типу данных сведения о населении.

    Args:
        list_of_tables (list whith pandas.DataFrames as  items ): если таблиц больше одной на странице то нужно указать ее индекс
        table_index (int): index of table

    Returns:
        _type_: pandas.DataFrame
    """
    # формируем сырой DataFrame
    df = list_of_tables[table_index].iloc[:,[1,3]] 
    # укорачиваем название столбца 'Население...'
    # чтобы не заморачиваться со старым и новым именем столбца просто
    # присвоим новые назщвания столбцам
    df.columns = ['Город', 'Население']
    # удаляем пробелы
    df['Население'] = df['Население'].apply(lambda x: x.replace(' ','')) 
    # приводим к числовому типу данных
    df['Население'] = df['Население'].astype('int64')
    return df

In [8]:
# код для получения списка городов-милионников для задания №

# вставляем оригинальную ссылку из Википедии
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России')
# приводим в читабельный вид
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
# получаем списао таблиц на страницу
tables_m = pd.read_html(url)
# сохраняем в кортеж с названиями стран в переменную 
million_cities = tuple(tables_m[1]['Город'])
#million_cities_df = get_df_from_viki(tables_m,7)

In [9]:

# вставляем оригинальную ссылку из Википедии на стараницу с данными о городах России
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города_России')
# приводим в читабельный вид
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
# получаем списао таблиц на страницу
tables = pd.read_html(url)

# сохраняем в кортеж с названиями стран в переменную 
# крупные города 500 тысяч до 1 миллиона человек
large_cities_500_1000 = tuple(tables[7]['Город'])
large_cities_500_1000_df = get_df_from_viki(tables,7)

# крупные города от 250 до 500 тысяч человек
large_cities_250_500 = tuple(tables[8]['Город'])
large_cities_250_500_df = get_df_from_viki(tables,8)

# большие города от 100 до 250 тысяч человек 
big_cities = tuple(tables[9]['Город'])
big_cities_df = get_df_from_viki(tables,9)

# средние города от 50 до 100 тысяч человек
middle_cities = tuple(tables[10]['Город'])
middle_cities_df = get_df_from_viki(tables,10)

# средние города от 50 до 100 тысяч человек
small_cities = tuple(tables[11]['Город'])
small_cities_df = get_df_from_viki(tables,11)


# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [10]:
# текст запроса 
# считаем количество строк в таблице vacancies
query_3_1 = f'''select
                --посчитаем все сторки
                count(*)
                --в таблице с вакансиями
                from vacancies 
            '''

In [11]:
# результат запроса
df_3_1 = pd.read_sql_query(query_3_1, connection)
vacancies_total_amount = df_3_1.iloc[0,0]
print(f'Общее количество вакансий в базе: {vacancies_total_amount}')

Общее количество вакансий в базе: 49197


2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [12]:
# текст запроса
# считаем количество строк в таблице employers
query_3_2 = f'''select
                --посчитаем все сторки
                count(*)
                --в таблице с работодателями
                from employers
            '''

In [13]:
# результат запроса
df_3_2 = pd.read_sql_query(query_3_2, connection)
employers_total_amount = df_3_2.iloc[0,0]
print(f'Общее количество работодателей в базе: {employers_total_amount}')

Общее количество работодателей в базе: 23501


3. Посчитате с помощью запроса количество регионов (таблица areas).

In [14]:
# текст запроса
# считаем количество строк в таблице areas
query_3_3 = f'''select
                --посчитаем все сторки
                count(*)
                --в таблице с регионами
                from areas
            '''

In [15]:
# результат запроса
df_3_3 = pd.read_sql_query(query_3_3, connection)
areas_total_amount = df_3_3.iloc[0,0]
print(f'Общее количество регионов в базе: {areas_total_amount}')

Общее количество регионов в базе: 1362


4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [16]:
# текст запроса
# считаем количество строк в таблице industries
query_3_4 = f'''select
                --посчитаем все сторки
                count(*)
                --в таблице с видами деяетельности
                from industries
            '''

In [17]:
# результат запроса
df = pd.read_sql_query(query_3_4, connection)
industries_total_amount = df.iloc[0,0]
print(f'Общее количество сфер деятельности в базе: {industries_total_amount}')

Общее количество сфер деятельности в базе: 294


***

In [18]:
# выводы по предварительному анализу данных
print(f'Общее количество вакансий в базе: {vacancies_total_amount}')
print(f'Общее количество работодателей в базе: {employers_total_amount}')
print(f'Общее количество регионов в базе: {areas_total_amount}')
print(f'Общее количество сфер деятельности в базе: {industries_total_amount}')
print()
print(f'Грубые средние оценки')
print(f'Количество вакансий на одного работодателя: {round(vacancies_total_amount/employers_total_amount)}')
print(f'Количество вакансий на один регион: {round(vacancies_total_amount/areas_total_amount)}')
print(f'Количество вакансий на одну сферу деятельности: {round(vacancies_total_amount/industries_total_amount)}')
print(f'Количество работодателей на один регион: {round(employers_total_amount/areas_total_amount)}')

Общее количество вакансий в базе: 49197
Общее количество работодателей в базе: 23501
Общее количество регионов в базе: 1362
Общее количество сфер деятельности в базе: 294

Грубые средние оценки
Количество вакансий на одного работодателя: 2
Количество вакансий на один регион: 36
Количество вакансий на одну сферу деятельности: 167
Количество работодателей на один регион: 17


**Общие выводы** по количественным данным и полученным соотношениям:<br>
*Количество вакансий больше чем работодателей в 2 раз, или 2 вакансии на одного работодателя,* это грубый вывод, требующей уточнения в дальнейшем, конечно это не означает что у каждого работодателя по 2 вакансии.<br>

Это означает что данные логичны и *потенциально* означает что у каждого работодателя есть хотя бы по одной вакансии, а может и нет, и значит необходимо *уточнить список работодателей у которых нет вакансий и в каких эти работодатели регионах и сферах деятельности.*<br>

Если таковые имеются то необходимо *учесть их долю* в общем количестве и в разрезе регионов.<br>

Судя по этим цифрам в каждом регионе по 17 работотелей, но в реальности это конечно же не так распределение работодателей по регионам *будет свидетельствовать о показателе деловой активности*, особенно если учесть уровни предлагаемых зарплат и количесвто сотрудников компании.<br>

Также интересно соотнести фактически показатель среднего числа вакансий на одного работодателя в общем к фактическому показателю среднего числа вакансий в каждом регионе. Число вакансий на одного работодателя может свидетельствовать: об уровне "текучки" кадров(хотя тут необходимо смотреть на соотношение к общему количеству сотрудников, и частоту закрытий/открытий одних и тех же вакансий на протяжении времени, т.е. добавляется временной разрез для каждой компании), развитии компании (т.е. создание новых подразделений)<br>

Если принять что в каждой сфере деятельности по 167 вакансий, то можно сказать что все они в равной мере испытывают потребность в кадрах, но это не так каждая сфера испытывает разную потребность в кадрах, и фактические значения это подтвердят. <br>


# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [19]:
# текст запроса
query_4_1 = f'''select
                --столбец с названиями регионов из таблицы регионов
                a.name as area,
                --считаем общее количество вакансий по столбцу id из таблицы вакансий   
                count(v.id) as total_vacancies
                --присоединямем к таблице вакансиями 
                from vacancies as v
                --таблицу с регионами по id региона (регион в котором размещена вакансия)
                join areas as a on v.area_id = a.id
                --группируем по названию региона
                group by a.name
                --сортировка по количеству вакансий в порядке убывания
                order by count(v.id) desc
            '''

In [20]:
# результат запроса
df_4_1 = pd.read_sql_query(query_4_1, connection)
display(df_4_1)

,area,total_vacancies
0,Москва,5333
1,Санкт-Петербург,2851
2,Минск,2112
3,Новосибирск,2006
4,Алматы,1892
...,...,...
764,Тарко-Сале,1
765,Новоаннинский,1
766,Бирск,1
767,Сасово,1


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [21]:
# текст запроса
query_4_2 = f'''select
                    --считаем все строки
                    count(*)
                --в таблице с вакансиями 
                from vacancies
                --где заполнена либо нижняя 
                where salary_from is not null
                    --либо верхняя граница зарплатной вилки
                    or salary_to is not null
            '''

In [22]:
# результат запроса
df = pd.read_sql_query(query_4_2, connection)
not_null_salary_amount = df.iloc[0,0]
print(f'Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой: {not_null_salary_amount}')

Количество вакансий у которых заполнено хотя бы одно из двух полей с зарплатой: 24073


2. 1. Дополнительно выясним у какого количества вакансий не  заполнено ни одно из двух полей с зарплатой.

In [23]:
query_4_2_2 = f'''select
                    --считаем все строки
                    count(*)
                --в таблицу с вакансиями
                from vacancies
                --где и верхняя и нижняя граница зарплатной вилки не заполнена
                where salary_from is null
                    and salary_to is null
            '''
df_4_2_2  = pd.read_sql_query(query_4_2_2, connection)
null_salary_amount = df_4_2_2.iloc[0,0]
print(f'Количество вакансий у которых не заполнено ни одно из двух полей с зарплатой: {null_salary_amount}')

Количество вакансий у которых не заполнено ни одно из двух полей с зарплатой: 25124


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [24]:
# текст запроса
query_4_3 = f'''select
                    --находим среднее значение нижней границы зарплатной вилки
                    avg(salary_from) as average_low_salary,
                    --находим среднее значение верхней границы зарплатной вилки
                    avg(salary_to) as average_high_salary
                --в таблице с вакансиями
                from vacancies
            '''

In [25]:
# результат запроса
df = pd.read_sql_query(query_4_3, connection)
average_low_salary = round(df.iloc[0,0])
average_high_salary = round(df.iloc[0,1])
print(f'Среднее значение нижней границы зарплатной вилки: {average_low_salary}')
print(f'Среднее значение верхней границы зарплатной вилки: {average_high_salary}')

Среднее значение нижней границы зарплатной вилки: 71065
Среднее значение верхней границы зарплатной вилки: 110537


3. 1. Дополнительно найдем средние значения для нижней и верхней границы "зарплатной вилки", а также диапазон "зарплатной вилки"  в разрезе опыта, отсортируем по нижней границе:

In [26]:
query_4_3_1 = f'''select
                    --выводим значение опыта работы
                    experience "Опыт работы",
                    --считаем средний нижний уровень и округляем до целых
                    round(avg(salary_from)) "Средний нижний уровень",
                    --считаем средний верхний уровень и округляем до целых
                    round(avg(salary_to)) "Средний верхний уровень",
                    --считаем разницу между средним верхним и нижним уровнем, округляем до целых
                    round(avg(salary_to)- avg(salary_from)) "Диапазон вилки"
                --из таблицы с вакансиями
                from vacancies 
                --группируем по опыту
                group by experience
                --сортируем по среднему нижнему уровню,округляем до целых
                order by round(avg(salary_from))
            '''

df_4_3_1 = pd.read_sql_query(query_4_3_1, connection)
display(df_4_3_1)

,Опыт работы,Средний нижний уровень,Средний верхний уровень,Диапазон вилки
0,Нет опыта,34699.0,56990.0,22292.0
1,От 1 года до 3 лет,64456.0,97436.0,32980.0
2,От 3 до 6 лет,112544.0,171974.0,59430.0
3,Более 6 лет,150156.0,197819.0,47664.0


3. 2. Дополнительно найдем средние значения для нижней и верхней границы зарплатной вилки в разрезе регионов - только города-миллионники, при условии что одно из полей с ЗП заполнены и сопоставим с количеством населения в этих городах:

In [27]:
query_4_3_2 = f'''select
                    --выбираем столбец с названием региона
                    a.name "Регион",
                    --находим среднее значение нижней границы зарплатной вилки,округляем до целых
                    round(avg(v.salary_from)) "Средний нижний уровень",
                    --находим среднее значение верхне границы зарплатной вилки,округляем до целых
                    round(avg(v.salary_to)) "Средний верхний уровень",
                    --считаем количество вакансий по столбцу id
                    count(v.id) "Количество вакансий", 
                    --считаем разницу между средним верхним и нижним уровнем, округляем до целых
                    round(avg(v.salary_to)- avg(v.salary_from)) "Диапазон вилки"
                  --к таблице вакансий
                  from vacancies v
                    --присоединяем таблицу с регионами по id региона (регион в котором размещена вакансия)
                    join areas a on v.area_id = a.id
                  --где заполнена либо нижняя 
                  where salary_from is not null
                    --либо верхняя граница зарплатной вилки
                    or salary_to is not null
                  --группируем по id региона
                  group by a.id
                  --фильтруем сгруппированные значения по спику городов-миллионников
                  having a.name in {million_cities}
                  --сортируем по среднему нижнему уровню, в порядку убывания,округляем до целых
                  order by round(avg(v.salary_from)) desc
                
            '''

df_4_3_2 = pd.read_sql_query(query_4_3_2, connection)
display(df_4_3_2)

,Регион,Средний нижний уровень,Средний верхний уровень,Количество вакансий,Диапазон вилки
0,Москва,113409.0,160594.0,1592,47185.0
1,Новосибирск,91272.0,137573.0,929,46301.0
2,Санкт-Петербург,89892.0,127249.0,1203,37356.0
3,Нижний Новгород,83734.0,126921.0,680,43187.0
4,Ростов-на-Дону,79391.0,119708.0,612,40317.0
5,Пермь,78932.0,130215.0,352,51282.0
6,Краснодар,78483.0,117144.0,693,38660.0
7,Екатеринбург,77489.0,120404.0,901,42915.0
8,Казань,76117.0,119942.0,628,43825.0
9,Омск,74898.0,107685.0,333,32787.0


In [28]:
# выгружаем из википедии данные о населении в городах миллионниках в переменную population
population = tables_m[1][['Город', 'Население, тыс. чел.  (на 1 января 2023 года)']]
# укорачиваем название столбца 'Население...'
population.rename(columns={'Население, тыс. чел.  (на 1 января 2023 года)':'Население,тыс. чел.'}, inplace=True)
# Для того чтобы сортировать по значениям
# 1. убираем пробелы в значениях
population['Население,тыс. чел.'] = population['Население,тыс. чел.'].apply(lambda x: x.replace(' ','')) 
# 2. присваиваем значениям числовой тип данных
population['Население,тыс. чел.'] = population['Население,тыс. чел.'].astype('int64')
# присоединяем полученный DF
new_df =  df_4_3_2.join(
    population.set_index('Город'),
    on='Регион',
    how='inner'
# и сортируем по населению в порядке убывания
).sort_values(by='Население,тыс. чел.', ascending=False)


In [29]:
# визуализируем в Plotly
# создаем фигуру с несколькими областями
fig = make_subplots(
    rows=3,
    cols=1,
    # устанавливаем пропорции
    row_heights=[3, 1,1],
    # задаем названия грфиков
    subplot_titles=(
        "Зависимость средней предлагаемой ЗП от города",
        "Зависимость количества вакансий от города",
        "Население по городам"
        )
    )

# добавляем данные по зарплатной вилке:верхний, нижний уровни и диапазон
fig.add_trace(go.Scatter(x = new_df['Регион'], y = new_df['Средний верхний уровень'], name='Верхний уровень ЗП'),1,1)
fig.add_trace(go.Scatter(x = new_df['Регион'], y = new_df['Средний нижний уровень'], name='Нижний уровень ЗП'),1,1)
fig.add_trace(go.Scatter(x = new_df['Регион'], y = new_df['Диапазон вилки'], name='Диапазон вилки'),1,1)
# добавляем данные по количесвтву вакансий
fig.add_trace(go.Scatter(x = new_df['Регион'], y = new_df['Количество вакансий'], name='Количество вакансий'),2,1)
fig.update_yaxes(range=[0, 1700], row=2, title='Количество вакансий')
fig.update_xaxes(row=2, title='Города-миллионники в порядке убывания населения')
# добавляем данные по населению
fig.add_trace(go.Scatter(x = new_df['Регион'], y = new_df['Население,тыс. чел.'], name='Население,тыс.чел.'),3,1)
fig.update_yaxes(range=[0, 14000], row=3, title='Население,тыс.чел.')
fig.update_xaxes(row=3, title='Города-миллионники в порядке убывания населения')

# настраиваем внешний вид
fig.update_layout(legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  hovermode="x",
                  margin=dict(l=10, r=10, t=40, b=10),
                  #title="Зависимость средней предлагаемой ЗП от города",
                  title_x=0.5,
                  xaxis_title="Города-миллионники в порядке убывания населения",
                  yaxis_title="ЗП, руб.",
                  height=1000
                  )
#настраиваем внешний вид интерактивных подсказок
fig.update_traces(hoverinfo="all", hovertemplate="Город: %{x}<br>Деньги: %{y}",row=1)                  
fig.update_traces(hoverinfo="all", hovertemplate="Город: %{x}<br>Количесвто: %{y}",row=2)
fig.update_traces(hoverinfo="all", hovertemplate="Город: %{x}<br>Население: %{y}",row=3)                  
fig.show()

![](pictures/million_cities_salary.png)

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [30]:
# текст запроса
query_4_4 = f'''select
                    --выводим тип графика
                    schedule schedule_type,
                    --выводими тип трудоустройства
                    employment employment_type,
                    --считаем количество вакансий по стобцу id
                    count(id) total_count    
                --из таблицы вакансий
                from vacancies 
                --группируем по типу графика, затем по типу трудоустройства
                group by schedule,employment
                --сортируем по количеству вакансий в порядку убывания
                order by count(id) desc    
            '''

In [31]:
# результат запроса
df = pd.read_sql_query(query_4_4, connection)
display(df)

,schedule_type,employment_type,total_count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [32]:
# текст запроса
query_4_5 = f'''select
                    --выводим значение опыта
                    experience,
                    --считаем вакансии по id
                    count(id) total_count   
                --из таблицы вакансий
                from vacancies 
                --группируем по опыту
                group by experience
                --сортируем по количеству вакансий в порядку убывания
                order by count(id) asc
            '''

In [33]:
# результат запроса
df_4_5 = pd.read_sql_query(query_4_5, connection)
display(df_4_5)

,experience,total_count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


5. 1. Добавим к предыдущему заданию фильтр: рассмотрим только те вакансии у которых нет данных о ЗП

In [34]:
# текст запроса
query_4_5_1 = f'''select
                    --выводим значение опыта
                    experience,
                    --считаем вакансии по id
                    count(id) null_salary_count   
                --из таблицы вакансий
                from vacancies 
                --где и верхняя и нижняя граница зарплатной вилки не заполнена
                where salary_from is null
                    and salary_to is null
                --группируем по опыту
                group by experience
                --сортируем по количеству вакансий в порядку убывания
                order by count(id) asc
            '''
# результат запроса
df_4_5_1 = pd.read_sql_query(query_4_5_1, connection)
display(df_4_5_1)            

,experience,null_salary_count
0,Более 6 лет,787
1,Нет опыта,2142
2,От 3 до 6 лет,8862
3,От 1 года до 3 лет,13333


In [35]:
# присоединим ко всем вакансиям вакансии без ЗП
# и сохраним в переменную 
experience_df = df_4_5.join(
    df_4_5_1.set_index('experience'),
    on='experience',
    how='inner'    
)
# переопределим названия столбцов 
experience_df.columns = ['Опыт', 'Вакансии все','Вакансии без ЗП']
# добавим столбец с долей вакансии без ЗП
experience_df['Доля вакансий без ЗП,%'] = round(experience_df['Вакансии без ЗП']/experience_df['Вакансии все']*100)
display(experience_df)

,Опыт,Вакансии все,Вакансии без ЗП,"Доля вакансий без ЗП,%"
0,Более 6 лет,1337,787,59.0
1,Нет опыта,7197,2142,30.0
2,От 3 до 6 лет,14511,8862,61.0
3,От 1 года до 3 лет,26152,13333,51.0


***

1. Подавляющее число работодателей рассматривает вариант сочетания: 
* типа графика - "полный день" и;
* типа трудоустройства - "полная занятость"

1. 1. Возможные причины в том, что работодатели **ищут работников на основное место работы**, и не заинтересованы в совместительстве;

2. Подавляющее число вакансий требует опыт работы от 1 года до 6 лет. Более половины соискателей имеют опыт от 1 до 3 лет. Вышеуказанное распределение может свидетельствовать о том, что люди набирающие опыт активно ищут новые возможности и лучшие места, соответственно они проще меняют работу, и "текучка" таких кадров больше. Также это может говорить о занитересованности в молодых, но уже опытных (чуть-чуть) специалистах, которые уже имеют представление о профессии и обучаемы в силу возраста.

3. Большинство вакансий сконцентрированно в крупных городах, что логично т.к. там больше работодателей.

4. Значение средних нижних и верхних границ "вилки" предлагаемых вакансий от 71 до 110 тысяч рублей. Это "средняя температура по больнице" которая дает глобальное представление о предложении на рынке труда - т.е. во что оценивают работодатели потенциальных работников. 

* 4. 1. **В разрезе опыта:** с увеличением опыта вилка сдвигается вправо, т.е. чем больше опыта, тем больше значения средних нижних и верхних границ "вилки". Диапазон разброса увеличивается от 0 до 6 лет опыта, а после опять уменьшается. Возможно это связано с тем что в течении 6 лет человек набирается опыта, каждый развивается с разной скоростью и соответственно возможен разный набор  компитенций, а после 6 лет уже набирается какой-то прогнозируемый набор компитенций, который работодатель может просчитать.

* 4. 2. **В разрезе региона**, а именно городов-миллионников РФ, при условии что хотя-бы одно из полей с ЗП заполнено: С уменьшением населения и уменьшаются верхние и нижние значения зарплатной вилки, также уменьшается и диапазон зарплатной вилки. Тенденция на уменьшение присутствует. Наблюдается большая корреляция с количеством вакансий, а не с населением конкретного города. **Примечательно что Новосибирск обогнал Санкт-Петербург** по значениям зарплатной вилки, хотя вакансий там меньше, возможные причины: Возможно компании побогаче, такое может быть если там открываются филиалы крупных компаний, но я предпочитаю думать что это по тому, что в Новосибирске преобладают более квалифицированные вакансии т.е. требуются управленцы, и уникальные специалисты.


5. Чуть более половины всех вакансий имеют пустые отметки о предлагаемой ЗП, что может свидетельствовать о:
 * гибком индивидуальном подходе, т.е. ЗП определяется по  результатам собедования, или;
 * ЗП - является конфиденциальной информацией и не выкладывается в открытый доступ, такая у  компании корпоративная политика;
 



# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [36]:
# текст запроса
query_5_1 = f'''select
                    --выведем название работодателя
                    e.name as employer,
                    --посчитаем количесво вакансий
                    count(v.id) as total_vacancies
                --к таблице вакансий
                from vacancies as v
                    --присоединим таблицу работодателй по id работодателя
                    join employers as e on v.employer_id = e.id
                --сгруппируем по названию руботодателя
                group by e.name
                --отсортируем по количеству ваканий в порядке убывания
                order by count(v.id) desc
                --выводим топ 5
                limit 5
            '''

In [37]:
# результат запроса
df_5_1 = pd.read_sql_query(query_5_1, connection)
display(df_5_1)
# сохраним количество вакансий яндекса в переменную
total_yandex_vacancies = df_5_1.iloc[0,1]

,employer,total_vacancies
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [38]:
# текст запроса
query_5_2 = f'''select
                    --выбираем столбец с названием региона    
                    a.name area,
                    --считаем количество работодателей
                    count(e.id) employers_count,
                    --считаем количесвтво вакансий
                    count(v.id) vac_count
                --к таблице с регионами
                from  areas a
                    --присоединяем таблицу с работодателями по id региона в котором расположен работодатель
                    left join employers e on a.id = e.area
                    --присоединяем таблицу с работодателями по id региона в котором расположена вакансия
                    left join vacancies v on a.id = v.area_id 
                --фильтруем по регионам в которых нет вакансий
                where v.name is null 
                --группируем по регионам(по id региона)
                group by a.id
                --сортируем по второму столбцу, т.е. по количеству работодателей
                order by 2 desc
            '''

In [39]:
# результат запроса
df_5_2 = pd.read_sql_query(query_5_2, connection)
display(df_5_2)
display(f'Регион где нет вакансий и самое болшое количество работодателей:{df_5_2.iloc[0,0]}')

,area,employers_count,vac_count
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


'Регион где нет вакансий и самое болшое количество работодателей:Россия'

3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [40]:
# текст запроса
query_5_3 = f'''select
                    --выбираем работодателей    
                    e.name employer,
                    --считаем уникальные значения по id региона
                    count(distinct v.area_id) areas_count    
                --к таблице работодателей 
                from  employers e 
                    --присоединяем левым join таблицу вакансий по id работодателя
                    left join vacancies v on e.id = v.employer_id 
                --группируем по id работодателя 
                group by e.id
                --сортируем по второму столбцу, т.е. по количеству уникальных id региона
                order by 2 desc
            '''

In [41]:
# результат запроса
df_5_3 = pd.read_sql_query(query_5_3, connection)
display(df_5_3)


,employer,areas_count
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
23496,Но Смокинг,0
23497,МАСТЕРСКАЯ CREED,0
23498,Dzing Finance,0
23499,КОРС,0


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [42]:
# текст запроса
query_5_4 = f'''select
                    --считаем уникальные id работодателя    
                    count(distinct e.id)    
                --к таблице работодателей
                from  employers e 
                    --присоединяем левым join таблицу employers_industries по id работодателя
                    left join employers_industries ei on e.id = ei.employer_id 
                --фильтруем значения, оставляеме только тех работодалей
                --у которых не заполнена сфера деятельности
                where ei.industry_id is null
             '''

In [43]:
# результат запроса
df_5_4 = pd.read_sql_query(query_5_4, connection)
display(f'Количество работодателей у которых не указана сфера деятульности:{df_5_4.iloc[0,0]}')

'Количество работодателей у которых не указана сфера деятульности:8419'

5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [44]:
# текст запроса
query_5_5 = f'''select
                    --название компании
                    e.name    
                --к таблице связке работодатель-сферая деятельности
                from employers_industries ei 
                    --присоединяем таблицу работодателей по id работотдателя
                    join employers e on ei.employer_id = e.id 
                --группируем по имени работодателя
                group by e.name
                --фильтруем по количеству видов деятельности равному четырем
                having count(ei.industry_id) = 4
                --сортируем в алфавитном порядку
                order by e.name
                --выводим 3 место
                offset 2 limit 1
             '''

In [45]:
# результат запроса
df_5_5 = pd.read_sql_query(query_5_5, connection)
display(f'Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которой указано четыре сферы деятельности:{df_5_5.iloc[0,0]}')


'Название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которой указано четыре сферы деятельности:2ГИС'

6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [46]:
# текст запроса
query_5_6 = f'''select
                    --считаем количество работодателей 
                    count(ei.employer_id)    
                --к таблице связке работодатель-сферая деятельности
                from employers_industries ei 
                    --присоединяем таблицу сфер деятельности  по id сферы деятельности
                    join industries i on ei.industry_id = i.id 
                --фильтруем по сфере деятельности
                where i.name like 'Разработка программного обеспечения'
                
             '''

In [47]:
# результат запроса
df_5_6 = pd.read_sql_query(query_5_6, connection)
count_it_vac = df_5_6.iloc[0,0]
display(f'Количество работодателей у которых в качестве сферы деятельности указана "Разработка программного обеспечения":{count_it_vac}')

'Количество работодателей у которых в качестве сферы деятельности указана "Разработка программного обеспечения":3553'

6. 1. Дополнительно вычислим долю работодателей у которых в качестве сферы деятельности<br> указана "Разработка программного обеспечения"

In [48]:
display(f'Доля работодателей у которых в качестве сферы деятельности указана "Разработка программного обеспечения":{round(count_it_vac/employers_total_amount*100)}%')

'Доля работодателей у которых в качестве сферы деятельности указана "Разработка программного обеспечения":15%'

7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.


Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [49]:
# код для получения списка городов-милионников

# вставляем оригинальную ссылку из Википедии
path_common = req.urlparse('https://ru.wikipedia.org/wiki/Города-миллионеры_России')
# приводим в читабельный вид
url = path_common.scheme + '://' + req.quote(path_common.netloc) +  req.quote(path_common.path) + '#'+ req.quote(path_common.fragment)
# получаем списао таблиц на страницу
tables = pd.read_html(url)
# сохраняем в кортеж с названиями стран в переменную 
million_cities = tuple(tables[1]['Город'])


In [50]:
# текст запроса
query_5_7 = f'''select
                    --название региона 
                    a.name "регион",
                    --количество вакансий в нем
                    count(v.id) "количество вакансий"
                --к таблице вакансий       
                from vacancies v 
                    --присоединяем таблицу работодателей по id работодателя
                    join employers e on v.employer_id = e.id
                    --присоединяем таблицу регионов по id региона
                    join areas a on v.area_id = a.id
                --фильтарция по имени работодателя
                where e.name like 'Яндекс'
                --группировка по регионам
                group by a.id
                --фильтарция только городов миллионников
                having a.name in {million_cities}
                
                union all --объединяем 
                
                -- добавляем строчку с общим количеством вакансий Яндекса
                select
                    'Total',
                    --считаем общее количесвто вакансий Яндекса
                    count(v.id)        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                    and a.name in {million_cities}
                
                --сортируем по убыванию количества вакансий 
                order by "количество вакансий"
                
             '''

In [51]:
# результат запроса
df_5_7 = pd.read_sql_query(query_5_7, connection)
display(df_5_7)

,регион,количество вакансий
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

7. 1. Дополнительно выясним сколько вакансий "Янекс" разместил по всей стране". Сгруппируем вакансии группам городов в зависимости от количества населения:

* *Крупнейшие* —  с населением свыше 1 миллиона человек
* *Крупные* —  от 250 тысяч человек до 1 миллиона человек (в том числе подкатегории от 250 до 500 тысяч и от 500 тысяч до 1 миллиона человек)
* *Большие* —  от 100 до 250 тысяч человек
* *Средние* —  от 50 до 100 тысяч человек
* *Малые* —  до 50 тысяч человек

In [52]:
all_rus_cities = million_cities+large_cities_500_1000+large_cities_250_500+big_cities+middle_cities+small_cities


In [53]:
# текст запроса
query_5_7_1 = f'''
                -- добавляем строчку с общим количеством вакансий Яндекса в городах-миллионниках
                select
                    'Крупнейшие,более 1 миллиона человек'"Тип города по населению",
                    count(v.id) "количество вакансий"
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {million_cities}
                
                union all
                -- добавляем строчку с общим количеством вакансий Яндекса в Крупных горохда 500000-1000000 человек
                
                select
                    'Крупные 500000-1000000 человек',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {large_cities_500_1000}
                        
                union all
                -- добавляем строчку с общим количеством вакансий Яндекса в Крупных городах 250000-500000 человек
                
                select
                    'Крупные 250000-500000 человек',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {large_cities_250_500}
                        
                union all
                -- добавляем строчку с общим количеством вакансий Яндекса в Больших города 250000-500000 человек
                
                select
                    'Большие, 100000-250000 человек',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {big_cities}
                
                union all
                -- добавляем строчку с общим количеством вакансий Яндекса в Средних городах 250000-500000 человек
                
                select
                    'Средние,50000-100000 человек',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {middle_cities}
                
                union all 
                -- добавляем строчку с общим количеством вакансий Яндекса в малых городах до 50000 человек   
                
                select
                    'Малые, до 50000 человек',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {small_cities}
                        
                union all    
                -- добавляем строчку с общим количеством вакансий Яндекса в российских городах 
                
                select
                    'Всего в российских городах',
                    count(v.id)
                        
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                        and a.name in {all_rus_cities}
                        
                union all    
                -- добавляем строчку с общим количеством вакансий Яндекса 
                
                select
                    'Всего вакансий Яндекса',
                    {total_yandex_vacancies}
                    
                --сортируем по убыванию количества вакансий         
                order by "количество вакансий" desc
                '''

In [54]:
df_5_7_1 = pd.read_sql_query(query_5_7_1, connection)
display(df_5_7_1)

,Тип города по населению,количество вакансий
0,Всего вакансий Яндекса,1933
1,Всего в российских городах,1797
2,"Крупнейшие,более 1 миллиона человек",485
3,"Большие, 100000-250000 человек",474
4,Крупные 250000-500000 человек,422
5,Крупные 500000-1000000 человек,405
6,"Средние,50000-100000 человек",11
7,"Малые, до 50000 человек",0


In [55]:
display(f'{round(df_5_7_1.iloc[1,1]/total_yandex_vacancies*100)}% вакансий Яндекса в России')
display(f'{round(df_5_7_1.iloc[2,1]/df_5_7_1.iloc[1,1]*100)}% российских вакансий Яндекса в городах миллионниках')
display(f'{round(df_5_7_1.iloc[3,1]/df_5_7_1.iloc[1,1]*100)}% российских вакансий Яндекса в крупных городах (500 тыс.-1 млн. чел.)')
display(f'{round(df_5_7_1.iloc[4,1]/df_5_7_1.iloc[1,1]*100)}% российских вакансий Яндекса в крупных городах (250-500 тыс.чел.)')
display(f'{round(df_5_7_1.iloc[5,1]/df_5_7_1.iloc[1,1]*100)}% российских вакансий Яндекса в больших городах (100-250 тыс.чел.)')
display(f'{round(df_5_7_1.iloc[6,1]/df_5_7_1.iloc[1,1]*100)}% российских вакансий Яндекса в средних городах (50-100 тыс.чел.)')

'93% вакансий Яндекса в России'

'27% российских вакансий Яндекса в городах миллионниках'

'26% российских вакансий Яндекса в крупных городах (500 тыс.-1 млн. чел.)'

'23% российских вакансий Яндекса в крупных городах (250-500 тыс.чел.)'

'23% российских вакансий Яндекса в больших городах (100-250 тыс.чел.)'

'1% российских вакансий Яндекса в средних городах (50-100 тыс.чел.)'

7. 2. Посмотрим как Яндекс раскинул свои вакансии по нашей необъятной Родине

In [76]:
# Сведения о географических кооординатах возьмем от сюда
cities = pd.read_csv('city.csv')
# Для некоторых городов федерального значения значения не заполнены 
# приравняем эти названия к названию региона
cities.loc[cities['city'].isna(), 'city'] = cities.loc[cities['city'].isna(), 'region']
                   

In [77]:
query_5_7_2 = f'''
                select
                    --название региона 
                    a.name "регион",
                    --количество вакансий в нем
                    count(v.id) "количество вакансий"
                --к таблице вакансий       
                from vacancies v 
                    --присоединяем таблицу работодателей по id работодателя
                    join employers e on v.employer_id = e.id
                    --присоединяем таблицу регионов по id региона
                    join areas a on v.area_id = a.id
                --фильтарция по имени работодателя
                where e.name like 'Яндекс'
                --группировка по регионам
                group by a.id
                --фильтарция только городов миллионников
                having a.name in {million_cities}
                '''
df_5_7_2 = pd.read_sql_query(query_5_7_2, connection)

# присоединяем полученный DF
mil_df =  df_5_7_2.join(
    cities.set_index('city'),
    on='регион',
    how='left'
# и сортируем по количеству вакансий
).sort_values(by='количество вакансий', ascending=False)

In [78]:
query_5_7_3 = f'''
                select
                    --название региона 
                    a.name "регион",
                    --количество вакансий в нем
                    count(v.id) "количество вакансий"
                --к таблице вакансий       
                from vacancies v 
                    --присоединяем таблицу работодателей по id работодателя
                    join employers e on v.employer_id = e.id
                    --присоединяем таблицу регионов по id региона
                    join areas a on v.area_id = a.id
                --фильтарция по имени работодателя
                where e.name like 'Яндекс'
                --группировка по регионам
                group by a.id
                --фильтарция только городов миллионников
                having a.name in {large_cities_500_1000}
                '''
df_5_7_3 = pd.read_sql_query(query_5_7_3, connection)
# присоединяем полученный DF
large_cities_500_1000_df =  df_5_7_3.join(
    cities.set_index('city'),
    on='регион',
    how='left'
# и сортируем по количеству вакансий
).sort_values(by='количество вакансий', ascending=False)

In [79]:
query_5_7_4 = f'''
                select
                    a.name "регион",
                    count(v.id) "количество вакансий"     
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                group by a.id
                having a.name in {large_cities_250_500}
                '''
df_5_7_4 = pd.read_sql_query(query_5_7_4, connection)
# присоединяем полученный DF
large_cities_250_500_df =  df_5_7_4.join(
    cities.set_index('city'),
    on='регион',
    how='left'
# и сортируем по количеству вакансий
).sort_values(by='количество вакансий', ascending=False)

In [80]:
query_5_7_5 = f'''
                select
                    a.name "регион",
                    count(v.id) "количество вакансий"     
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                group by a.id
                having a.name in {big_cities}
                '''
df_5_7_5 = pd.read_sql_query(query_5_7_5, connection)
# присоединяем полученный DF
big_cities_df =  df_5_7_5.join(
    cities.set_index('city'),
    on='регион',
    how='left'
# и сортируем по количеству вакансий
).sort_values(by='количество вакансий', ascending=False)

In [81]:
query_5_7_6 = f'''
                select
                    a.name "регион",
                    count(v.id) "количество вакансий"     
                from vacancies v 
                    join employers e on v.employer_id = e.id
                    join areas a on v.area_id = a.id
                where e.name like 'Яндекс'
                group by a.id
                having a.name in {middle_cities}
                '''
df_5_7_6 = pd.read_sql_query(query_5_7_6, connection)
# присоединяем полученный DF
middle_cities_df =  df_5_7_6.join(
    cities.set_index('city'),
    on='регион',
    how='left'
# и сортируем по количеству вакансий
).sort_values(by='количество вакансий', ascending=False)

In [82]:
# визуализируем
# Отражаем города миллионники
fig = go.Figure(go.Scattermapbox(name='Города миллионники',
                                 lat=mil_df['geo_lat'], 
                                 lon=mil_df['geo_lon'],
                                 text='Город: '+mil_df['регион']+'.<br>' \
                                     +'Количество вакансий '+mil_df['количество вакансий'].astype(str)+' шт.',
                                 marker=dict(size=mil_df['количество вакансий'])))                                             
# Добавляем крупные города 500000-1000000 человек
fig.add_trace(go.Scattermapbox(name='Крупные города 500 тыс.- миллион человек',
                               lat=large_cities_500_1000_df['geo_lat'], 
                               lon=large_cities_500_1000_df['geo_lon'],
                               text='Город: '+large_cities_500_1000_df['регион']+'.<br>' \
                                    +'Количество вакансий '+large_cities_500_1000_df['количество вакансий'].astype(str)+' шт.',
                               marker=dict(size=large_cities_500_1000_df['количество вакансий']))) 

# Добавляем крупные города 250000-500000 человек
fig.add_trace(go.Scattermapbox(name='Крупные города 250-500 тыс.человек',
                               lat=large_cities_250_500_df['geo_lat'], 
                               lon=large_cities_250_500_df['geo_lon'],
                               text='Город: '+large_cities_250_500_df['регион']+'.<br>' \
                                    +'Количество вакансий '+large_cities_250_500_df['количество вакансий'].astype(str)+' шт.',
                               marker=dict(size=large_cities_250_500_df['количество вакансий']))) 

# Добавляем большие города 100000-250000 человек
fig.add_trace(go.Scattermapbox(name='Большие города 100-250 тыс.человек',
                               lat=big_cities_df['geo_lat'], 
                               lon=big_cities_df['geo_lon'],
                               text='Город: '+big_cities_df['регион']+'.<br>' \
                                    +'Количество вакансий '+big_cities_df['количество вакансий'].astype(str)+' шт.',
                               marker=dict(size=big_cities_df['количество вакансий']))) 


# Добавляем средние города 50000-100000 человек
fig.add_trace(go.Scattermapbox(name='Средние города 50-100 тыс.человек',
                               lat=middle_cities_df['geo_lat'], 
                               lon=middle_cities_df['geo_lon'],
                               text='Город: '+middle_cities_df['регион']+'.<br>' \
                                    +'Количество вакансий '+middle_cities_df['количество вакансий'].astype(str)+' шт.',
                               marker=dict(size=middle_cities_df['количество вакансий']))) 
# настраиваем центр карты                   
map_center = go.layout.mapbox.Center(lat=(cities['geo_lat'].max()+cities['geo_lat'].min())/2, 
                                     lon=(cities['geo_lon'].max()+cities['geo_lon'].min())/2)

fig.update_layout(title='Распределение вакансий Яндекса по городам России',
                  title_x=0.5,
                  legend_orientation="h",
                  legend=dict(x=.5, xanchor="center"),
                  mapbox_style="open-street-map",
                  mapbox=dict(center=map_center, zoom=2))


fig.show()

![](pictures/yandex_cities.png)

Выводы по анализу работодателей:

1. В топе по количеству вакансий технолочичные компании:такие как Яндекс, Ростелеком, СБЕР - это может свидетельствовать о том , для них портал HH.ru является хорошим источником для поиска персонала, и соискатели которые там обитают их устраивают по корпоративным требованиям.

2. На HH.ru больше всего работодателей без вакансий из России. Возможные причины: 

* портал изначально российский;
* В России больше всего работодателей с полностью укомплектованным штатом сотрудников в данный момент;

3. Доля IT компаний на HH.ru составляет 15% что является существенной долей в структуре клиентов компании (HH.ru), что обязывает учитывать интиресы индустирии (IT) и при работе, развитии сервиса.

4. Яндекс раскинул свои щупальца по всей стране(России). 93% вакансий  Яндекса в России. Российские вакансии распределены по городам-миллионикам(27%), крупным городам до 1 млн.чел (26%),крупным городам до 500 тыс.чел.(23%), большим городам (23%). В средних городах 1 % В малых городах Яндекс не размещал свои вакансии, это логично ведь из малых городов соискатели будут искать вакансии в ближайших средних и больших городах. Либо они от туда уже уехали вообще. Основная масса вакансий сосредоточена в Европейской части России.



# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [64]:
# текст запроса
query_6_1 = f'''select
                    --считаем  по id
                    count(id)    
                --из таблицы вакансий
                from vacancies    
                --фильтрация по вхождению строки в название вакансии
                where name ilike '%data%' or name ilike '%данн%' 
             '''

In [65]:
# результат запроса
df_6_1 = pd.read_sql_query(query_6_1, connection)
total_ds_vac = df_6_1.iloc[0,0]
display(f'{total_ds_vac} вакансий имеют отношение к данным')
display(f'{round(total_ds_vac/vacancies_total_amount*100)}% вакансий от общей базы имеют отношение к данным')

'1771 вакансий имеют отношение к данным'

'4% вакансий от общей базы имеют отношение к данным'

2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [66]:
# текст запроса
query_6_2 = f'''select
                    --считаем  по id
                    count(id)    
                --из таблицы вакансий
                from vacancies    
                --фильтрация по вхождению строки 
                --для дата-сайентистов
                where (name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    --исключаем вакансии по HTML
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    --для специалистов уровня Junior
                    and (name ilike '%junior%'
                    or experience like 'Нет опыта'
                    or employment like 'Стажировка')  
             '''

In [67]:
# результат запроса
df_6_2 = pd.read_sql_query(query_6_2, connection)
junior_vac = df_6_2.iloc[0,0] 
display(f'{junior_vac} вакансий для начинающего дата-сайентиста')
display(f'{round(junior_vac/total_ds_vac*100)}% вакансий которые имеют отношение к данным для начинающего дата-сайентиста')


'51 вакансий для начинающего дата-сайентиста'

'3% вакансий которые имеют отношение к данным для начинающего дата-сайентиста'

3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [68]:
# текст запроса
query_6_3 = f'''select
                    count(id)    
                from vacancies    
                --фильтрация по вхождению строки 
                --для дата-сайентистов
                where (name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    --по ключевым навыкам
                    and key_skills is not null
                    and (key_skills ilike '%sql%'
                    or key_skills ilike '%postgres%')  
             '''

In [69]:
# результат запроса
df_6_3 = pd.read_sql_query(query_6_3, connection)
sql_skill_vac = df_6_3.iloc[0,0]
display(f'{sql_skill_vac} вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres')
display(f'{round(sql_skill_vac/total_ds_vac*100)}% вакансий которые имеют отношение к данным, в которых в качестве ключевого навыка указан SQL или postgres')

'201 вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres'

'11% вакансий которые имеют отношение к данным, в которых в качестве ключевого навыка указан SQL или postgres'

4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [70]:
# текст запроса
query_6_4 = f'''select
                    count(id)    
                from vacancies    
                --фильтрация по вхождению строки 
                --для дата-сайентистов
                where (name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    --по ключевым навыкам
                    and key_skills is not null
                    and key_skills ilike '%python%'
             '''
                    

In [71]:
# результат запроса
df_6_4 = pd.read_sql_query(query_6_4, connection)
python_skill_vac = df_6_4.iloc[0,0]
display(f'{python_skill_vac} вакансий для DS, в которых в качестве ключевого навыка указан Python')
display(f'{round(python_skill_vac/total_ds_vac*100)}% вакансий которые имеют отношение к данным, в которых в качестве ключевого навыка указан Python')

'351 вакансий для DS, в которых в качестве ключевого навыка указан Python'

'20% вакансий которые имеют отношение к данным, в которых в качестве ключевого навыка указан Python'

5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [72]:
# текст запроса
query_6_5 = f'''select
                    --считаем среднее количесвто ключевых навыков
                    avg(length(key_skills) - length(replace(key_skills, CHR(9), ''))+1)
                from vacancies    
                where name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or name like '%ML%' and name not like '%HTML%'
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%'
             '''

In [73]:
# результат запроса
df_6_5 = pd.read_sql_query(query_6_5, connection)
display(f'Среднее количесвтво ключевых навыков:{round(df_6_5.iloc[0,0],2)}')

'Среднее количесвтво ключевых навыков:6.41'

6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [74]:
# текст запроса
query_6_6 = f'''select
                    --опыт
                    experience,
                    --считаем среднее значение ЗП
                    ROUND(AVG(coalesce((salary_from + salary_to) / 2, salary_from, salary_to)), 0)    
                from vacancies    
                --фильтрация по вхождению строки 
                --для дата-сайентистов
                where (name ilike '%data scientist%'
                    or name ilike '%data science%'
                    or name ilike '%исследователь данных%'
                    or (name like '%ML%' and name not like '%HTML%')
                    or name ilike '%machine learning%'
                    or name ilike '%машинн%обучен%')
                    -- заполнено хотя бы одно из двух полей с зарплатой
                    and (salary_from is not null
                    or salary_to is not null)
                --группировка по опыту
                group by experience
             '''

In [75]:
# результат запроса
df_6_6 = pd.read_sql_query(query_6_6, connection)
display(df_6_6)

,experience,round
0,Нет опыта,74643.0
1,От 1 года до 3 лет,139675.0
2,От 3 до 6 лет,243115.0


***

**Выводы по предметному анализу:**

1. Есть вакансии для Дата-саентиста.4% вакансий от всей базы имеют отношение к данным

2. Вакансии для Дата-саентиста с опытом до одного года отсутствуют в базе не представлены, возможно это связано что опыт меньше года рассматривается работодателем как отсутствие опыта. Проще взять способного с теоретическими знаниями и заточить под свои задачи, чем переучивать криворуких и самонадеянных.

3. За первый год рабыты ЗП Дата-саентиста может вырасти почти в два раза. На реальной работе оттачиваются реально важные навыки и знания, лишняя теория отсеивается - так нарабатываются компетенции. Реальные проекты в портфолио дает возможность работодателю подтвердить наличие и оценить уровень реальных навыков

4. Среднее количество навыков Дата саентиста более 6 это означает что дата саентист должен обдадать комплексными знаниями и навыками и уметь сопрягать их.

5. 97% вакансий для уже опытного Дата-саентиста. Но есть вакансии и для совсем начинающих. Все хотят уже опытных работников

6. Среди ключевых навыков Python (20%) практически в 2 раза востребованее чем SQL или postgres (11%)


# Общий вывод по проекту

1. Общие выводы: Наблюдается прямая зависимость уровня ЗП от размеров городов по численности населения. т.е. чем больше население тем больше работодателей в этом регионе, и они конкурируют между собой. 

**Каких вакансий много?** - *около половины вакансий без указания какого либо уровня ЗП*. Только 30 % вакансий для людей без опыта не имеют данных о ЗП. *Для других категорий опыта(1-3 года, 3-6 лет, более 6 лет)- доля вакансий не имеющих данных о ЗП более 50%*.

Причиной тому может быть: 
* гибкий подход к назначению ЗП, т.е. по резульататам собеседования;
* конфиденциальность ЗП согласно корпоративной политике (в больших корпорациях бюрократия часто превыше всего, и здравого смысла в том числе);
* сложная структура начисления ЗП, т.е. есть фиксированный оклад и много разных надбавок, в результате чего ЗП от месяца к месяцу меняется, HR страхуются и не публикуют эти данные в самой вакансии, а доводят информацию в на этапе общения с кандидатом. Следует понимать что причины могут быть разными как в отдельной отрасли, в отдельном регионе, у отдельного работодателя
* уж совсем дикая версия - это дубликаты вакансий с ЗП)

При построении модели стоит учесть влияние вакансий с незаполненными данными о ЗП. 

2. Прогнозы: Без учета вакансий с незаполненными данными о ЗП, модель вероятно не будет корректной.

3. Варианты продолжения исследования: 

* вакансии с незаполненными данными о ЗП нуждаются в дополнительном исследования для установления причин отсутствия этих данных и принятия решения об их учете(удалить, заполнить, чем заполнить); в разрезе: опыта, работодателя, индустрии;
* провести исследование распределения всех вакансий, вакансий сферы IT, вакансий на позицию Data Scientist по городам России по образцу исследования компании "Яндекс"(Размер маркера: количество вакансий, цвет маркера: средняя ЗП)


